In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import statistics


from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization, Add, Activation, ZeroPadding2D, Input, AveragePooling2D, MaxPool2D
from tensorflow.keras import activations
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import categorical_crossentropy, sparse_categorical_crossentropy
from matplotlib.font_manager import FontProperties

from google.colab import files
import cv2
import sys
from tensorflow.keras.utils import to_categorical
import numpy as np
import math as m
!pip install XlsxWriter
import xlsxwriter

In [ ]:
batch_size = 64
(ds_train, ds_test), ds_info = tfds.load(
    'imagenette/160px-v2',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
class_types = 10


print(ds_info)
tfds.features.Image

In [24]:
IMG_SIZE = 100

resize_and_rescale = tf.keras.Sequential([
  keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  keras.layers.experimental.preprocessing.Rescaling(1)
])

In [25]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  image = tf.image.resize(image, (100, 100))
  return tf.cast(image, tf.float32) / 255 - 0.5, label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(64)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


In [26]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(64)

In [27]:
class CustomRandomNormal(tf.keras.initializers.Initializer):

  def __init__(self, mean, stddev):
    self.mean = mean
    self.stddev = stddev

  def __call__(self, shape, dtype=None, **kwargs):
    return 0.1 * tf.random.normal(
        shape, mean=self.mean, stddev=self.stddev, dtype=dtype)

  def get_config(self):  # To support serialization
    return {"mean": self.mean, "stddev": self.stddev}

In [28]:
def build_vgg(activation, dense_activation):

    inputs = Input(shape = (100, 100, 3))
    x = inputs
    #initializer = CustomRandomNormal(-1, 1)
    x = Conv2D(64, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(64, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = MaxPool2D(pool_size = (2, 2))(x)

    x = Conv2D(128, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(128, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = MaxPool2D(pool_size = (2, 2))(x)

    x = Conv2D(256, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(256, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(256, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = MaxPool2D(pool_size = (2, 2))(x)

    x = Conv2D(512, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(512, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(512, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = MaxPool2D(pool_size = (2, 2))(x)

    x = Conv2D(512, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(512, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = Conv2D(512, kernel_size = (3, 3), padding = 'same', activation = activation)(x)
    x = MaxPool2D(pool_size = (2, 2))(x)

    x = Flatten()(x)

    x = Dense(2048, activation = dense_activation)(x)
    x = Dense(2048, activation = dense_activation)(x)

    output = Dense(10, activation = 'softmax')(x)

    model = Model(inputs = [inputs], outputs = [output])

    return model


In [ ]:
ds_test

In [30]:
def step(X, y):
	# keep track of our gradients
	with tf.GradientTape() as tape:
		# make a prediction using the model and then calculate the
		# loss
		pred = model(X)
		loss = sparse_categorical_crossentropy(y, pred)

	# calculate the gradients using our tape and then update the
	# model weights
	grads = tape.gradient(loss, model.trainable_variables)
	opt.apply_gradients(zip(grads, model.trainable_variables))

	return grads

In [31]:
def rms_val(gradient):
  total = 0
  count = 0
  for grad in gradient:
    total += tf.square(grad)
    count += 1

  val = tf.sqrt(total/count)
  return val.numpy()

In [32]:
def signum(x):
  return tf.math.sign(x)

In [33]:
def identity(x):
  return x

In [34]:
def gcu(x):
  return x * tf.cos(x)

In [35]:
def bipolar(x):
  return (1-tf.exp(-x))/(1+tf.exp(-x))

In [36]:
def swish(x):
  return x * tf.sigmoid(x)

In [37]:
lrelu=tf.keras.layers.LeakyReLU(alpha=0.01)

In [38]:
Tanh=tf.tanh

In [39]:
absolute=tf.abs

In [40]:
swish=tf.keras.activations.swish

In [41]:
def lisht(x):
  return x*tf.tanh(x)

In [42]:
def softplus(x):
  return tf.math.softplus(x,name=None)

In [43]:
def gelu(x):
  return tf.keras.activations.gelu(
    x, approximate=False
)

In [44]:
def selu(x):
  return tf.keras.activations.selu(x)

In [45]:
def mish(x):
  return x * tf.tanh(softplus(x))


In [46]:
def elu(x):
  return tf.keras.activations.elu(x)

In [47]:
def prelu(x):
  alphas = 0.25
  pos = tf.nn.relu(x)
  neg = alphas * (x - abs(x)) * 0.5

  return pos + neg

In [48]:
def sine(x):
  return tf.sin(x)

In [49]:
def squ(x):
  return x*x+x

In [50]:
def monotonic(x):
  return x*x*x+x

In [51]:
def nonmonotonic(x):
  return x-x*x*x

In [52]:
def z2cosz(x):
  return x*x*tf.cos(x)

In [53]:
def ssu(x):
  pi=np.pi
  return pi*tf.experimental.numpy.sinc(x-pi)

In [54]:
def dsu(x):
  pi=np.pi
  return pi*(tf.experimental.numpy.sinc(x-pi)-tf.experimental.numpy.sinc(x+pi))/2

In [55]:
history_plot_a=[None]*23
history_plot_l=[None]*23
func_name=[None]*23

In [56]:
num_functions = 23
num_plots = num_functions
epochs=25
Trials=1

In [ ]:
j=1   #Vary j between 0 and 22 to choose different activation functions

if j==0:
  func=signum
  func_label="Signum"
if j==1:
  func=identity
  func_label="Identity"
if j==2:
  func=bipolar
  func_label="Bipolar"
if j==3:
  func='sigmoid'
  func_label="Sigmoid"
if j==4:
  func=Tanh
  func_label="Tanh"
if j==5:
  func=absolute
  func_label="Absolute"
if j==6:
  func=lisht
  func_label="LiSHT"
if j==7:
  func=softplus
  func_label="Softplus"
if j==8:
  func='relu'
  func_label="ReLU"
if j==9:
  func=lrelu
  func_label="LeakyReLU"
if j==10:
  func=gelu
  func_label="GELU"
if j==11:
  func=selu
  func_label="SELU"
if j==12:
  func=mish
  func_label="Mish"
if j==13:
  func=swish
  func_label="Swish"
if j==14:
  func=elu
  func_label="ELU"
if j==15:
  func=prelu
  func_label="PReLU"
if j==16:
  func=sine
  func_label="SU"
if j==17:
  func=squ
  func_label="SQU"
if j==18:
  func=nonmonotonic
  func_label="NMC"
if j==19:
  func=z2cosz
  func_label="z2Cosz"
if j==20:
  func=ssu
  func_label="SSU"
if j==21:
  func=gcu
  func_label="GCU"
if j==22:
  func=dsu
  func_label="DSU"
print(j+1,func_label)
net_model = build_vgg(func,func)
net_model.compile(optimizer = RMSprop(learning_rate=1e-5) , loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history_a = net_model.fit(ds_train,epochs=epochs,validation_data = ds_test)
results = net_model.evaluate(ds_test)

print("Test loss=",results[0])
print("Test accuracy=",results[1])


In [ ]:
print("Simulation Complete!")
